# This is a sample notebook to generate synthetic data of overlapping particles

In [7]:
# import some necessary python modules 
import numpy as np
import yaml
import pandas as pd
import random 
import os
import glob 
import tqdm
from scipy.ndimage import gaussian_filter
import math
import scipy
from pathlib import Path
import matplotlib.pyplot as plt
plt.style.use('classic')

In [9]:
# import the user defined modules 
from image_generate_modules import gen_path, image_generator,  pick_xy_with_min_distance 
from utils.config_utils import load_config

In [10]:
# declare necessary cirectory architecture
# ........... build the path architecture ........ 
cwd = Path().resolve() 
Parent = cwd.parent
# set file path architecture 
path_data = load_config(str(cwd.parent / 'configs/config.yaml'))
# path to synthetic training data 
train_data_syn = cwd.parent /path_data['Images']['FirstTrain_syn']
# path to the heatmaps (parquet and the .npy)
parquet_path = cwd.parent /path_data['Heatmaps']['Parquet_Transfer']
heatmap_data_syn = cwd.parent /path_data['Heatmaps']['FirstTrain_syn']

# Location to store the training images 
syn_img_folder = str(train_data_syn)
syn_heatmap_folder = str(heatmap_data_syn )

# location to store the 

In [ ]:
# define the frame dimensions
X_size = 128
Y_size = 128

N_images = 10           # number of images
nbParticles = 500       # The approximate number of particles 
PtoB_ratio  = 17/512    # PtoB_ratio maps the particle size in microns to the image in the pixels 
Packing_fraction = 0.5  # packing fraction of the particles 

# create path locations for saveing the synthetic images
path_list = gen_path(syn_img_folder, N_images) 

# stores data of the particle postion as 2D binary mask
H_list = []

# create a bunch of images  
for i in tqdm.tqdm(range(N_images)) :
    X_, Y_, H = image_generator(path_list[i], [X_size, Y_size], nbParticles , PtoB_ratio=PtoB_ratio, packing_frac = Packing_fraction)
    H_list.append(H.flatten())

In [16]:
# save the 2D logistic map parquet format ... serves as a metadata ....
parquet_name = 'temp.parquet'
H_val = pd.DataFrame()
H_val['H'] = H_list
H_val.to_parquet(str(parquet_path /'temp.parquet'))

In [19]:
# create heatmaps

# create path locations for saveing the synthetic image heatmaps 
path_list = gen_path(syn_heatmap_folder, N_images)

# run the loop for developing the synthetic image heatmaops 
for i in tqdm.tqdm(range(len(H_val))):
    sigma = 0.7  # Spread of the Gaussian
    
    # resize the data into a 2D binary mask
    binary_mask =  H_val['H'].iloc[i].reshape(X_size, Y_size) # Should be shape (128, 128) with 1s at centers
    # create heatmaps 
    heatmap = gaussian_filter(binary_mask.astype(np.float32), sigma=sigma)
    
    # arrest the values of teh htmap between 0 and 1 
    heatmap = np.clip(heatmap, 0, 1)
    
    # save as a 2D numpy array
    np.save(path_list[i]+'.npy', heatmap)

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:52<00:00, 189.45it/s]
